In [1]:
!pip install keras-tuner

     |████████████████████████████████| 61kB 5.1MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=7b1c8555ac1a65ad35557c68d01e40306669ef4f7979d79e9836ac2cb9a30862
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=b67ba4fb722753c41d5577f27e5d99255f26aecf5da536c0825892dcecebc586
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
print(tf.__version__)

2.3.0


In [4]:
fashion_mnist=keras.datasets.fashion_mnist

In [5]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [6]:
train_images=train_images/255.0
test_images=test_images/255.0

In [7]:
train_images[0].shape

(28, 28)

In [8]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [9]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [10]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [11]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [12]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Epoch 1/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.3787 - accuracy: 0.8634 - val_loss: 0.2751 - val_accuracy: 0.8992
Epoch 2/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.2330 - accuracy: 0.9134 - val_loss: 0.2436 - val_accuracy: 0.9115
Epoch 3/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1701 - accuracy: 0.9362 - val_loss: 0.2550 - val_accuracy: 0.9095


Epoch 1/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.4037 - accuracy: 0.8542 - val_loss: 0.2922 - val_accuracy: 0.8908
Epoch 2/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.2620 - accuracy: 0.9037 - val_loss: 0.2877 - val_accuracy: 0.8982
Epoch 3/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.2097 - accuracy: 0.9216 - val_loss: 0.2858 - val_accuracy: 0.9040


Epoch 1/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.4879 - accuracy: 0.8207 - val_loss: 0.3962 - val_accuracy: 0.8563
Epoch 2/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.3774 - accuracy: 0.8601 - val_loss: 0.3968 - val_accuracy: 0.8610
Epoch 3/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.3397 - accuracy: 0.8719 - val_loss: 0.3735 - val_accuracy: 0.8718


Epoch 1/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.4960 - accuracy: 0.8272 - val_loss: 0.3996 - val_accuracy: 0.8465
Epoch 2/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.3660 - accuracy: 0.8667 - val_loss: 0.3859 - val_accuracy: 0.8635
Epoch 3/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.3363 - accuracy: 0.8761 - val_loss: 0.3519 - val_accuracy: 0.8732


Epoch 1/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.4199 - accuracy: 0.8493 - val_loss: 0.3210 - val_accuracy: 0.8815
Epoch 2/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.2744 - accuracy: 0.9007 - val_loss: 0.2751 - val_accuracy: 0.8997
Epoch 3/3
1688/1688 [==============================] - 8s 4ms/step - loss: 0.2223 - accuracy: 0.9178 - val_loss: 0.3021 - val_accuracy: 0.8980


INFO:tensorflow:Oracle triggered exit


In [13]:
model=tuner_search.get_best_models(num_models=1)[0]

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 48)        480       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 48)        20784     
_________________________________________________________________
flatten (Flatten)            (None, 27648)             0         
_________________________________________________________________
dense (Dense)                (None, 96)                2654304   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                970       
Total params: 2,676,538
Trainable params: 2,676,538
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1671 - accuracy: 0.9378 - val_loss: 0.2546 - val_accuracy: 0.9133
Epoch 5/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1181 - accuracy: 0.9561 - val_loss: 0.2517 - val_accuracy: 0.9163
Epoch 6/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0798 - accuracy: 0.9717 - val_loss: 0.3246 - val_accuracy: 0.9137
Epoch 7/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0539 - accuracy: 0.9802 - val_loss: 0.3568 - val_accuracy: 0.9108
Epoch 8/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0355 - accuracy: 0.9874 - val_loss: 0.3983 - val_accuracy: 0.9107
Epoch 9/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0289 - accuracy: 0.9893 - val_loss: 0.4931 - val_accuracy: 0.9110
Epoch 10/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0256 - accuracy: 0.9909 - val_loss: 0.5075 - val_accuracy